In [1]:
import json 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from collections import Counter


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_FEMALE_1610_1.json') as f:
    data = json.load(f)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_FEMALE_1610_1.json', 'r') as file:
    female_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_MALE_1610_1.json', 'r') as file:
    male_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NEUT_1610_1.json', 'r') as file:
    neutral_data_model1 = json.load(file)

with open('//Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_FEMALE_1610_1.json', 'r') as file:
    female_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_MALE_1610_1.json', 'r') as file:
    male_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NEUT_1610_1.json', 'r') as file:
    neutral_data_model2 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model1 = json.load(file)

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model2 = json.load(file)

In [3]:
def aggregate_data(data, gender, model):
    aggregated_data = []

    for prompt, adjectives in data.items():
        for adjective, details in adjectives.items():
            aggregated_data.append({
                "Prompt": prompt,
                "Descriptor": adjective,
                "Logit": details["logit"],
                "Antonym": details["antonym"],
                "Antonym Logit": details["antonym_logit"],
                "Favored": details["favored"],
                "Gender": gender,
                "Model": model  # Add model label
            })

    return aggregated_data

female_aggregated_model1 = aggregate_data(female_data_model1, "Female", "LLaMA-3-8b")
male_aggregated_model1 = aggregate_data(male_data_model1, "Male", "LLaMA-3-8b")
neutral_aggregated_model1 = aggregate_data(neutral_data_model1, "Neutral", "LLaMA-3-8b")

female_aggregated_model2 = aggregate_data(female_data_model2, "Female", "LLaMA-2-7b")
male_aggregated_model2 = aggregate_data(male_data_model2, "Male", "LLaMA-2-7b")
neutral_aggregated_model2 = aggregate_data(neutral_data_model2, "Neutral", "LLaMA-2-7b")


all_data = (female_aggregated_model1 + male_aggregated_model1 + neutral_aggregated_model1 +
            female_aggregated_model2 + male_aggregated_model2 + neutral_aggregated_model2)

df = pd.DataFrame(all_data)

diagnoses = [
    "schizophrenia", 
    "depression", 
    "generalised anxiety disorder", 
    "bipolar disorder", 
    "borderline personality disorder", 
    "a mental illness", 
    "antisocial personality disorder", 
    "post-traumatic stress disorder", 
    "obsessive-compulsive disorder", 
    "dissociative identity disorder"
]

def extract_diagnosis(prompt):
    for diagnosis in diagnoses:
        if diagnosis in prompt.lower():  
            return diagnosis
    return None  

reshaped_data = []

for _, row in df.iterrows():
    diagnosis = extract_diagnosis(row['Prompt'])  
    
    reshaped_data.append({
        'Prompt': row['Prompt'],  
        'adjective': row['Descriptor'],
        'logit': row['Logit'],
        'label': 'negative',  
        'Gender': row['Gender'],
        'Model': row['Model'],
        'diagnosis': diagnosis  
    })
    
    reshaped_data.append({
        'Prompt': row['Prompt'],  
        'adjective': row['Antonym'],
        'logit': row['Antonym Logit'],
        'label': 'positive',  
        'Gender': row['Gender'],
        'Model': row['Model'],
        'diagnosis': diagnosis  
    })

reshaped_df = pd.DataFrame(reshaped_data)

severity_mapping = {
    "schizophrenia": "severe",
    "depression": "not severe",
    "generalised anxiety disorder": "not severe",
    "bipolar disorder": "severe",
    "borderline personality disorder": "severe",
    "a mental illness": "not severe",  
    "antisocial personality disorder": "severe",
    "post-traumatic stress disorder": "not severe",
    "obsessive-compulsive disorder": "not severe",
    "dissociative identity disorder": "severe"
}

def add_severity_label(row):
    diagnosis = row['diagnosis']
    return severity_mapping.get(diagnosis, "unknown")  

reshaped_df['severity'] = reshaped_df.apply(add_severity_label, axis=1)

aggregated_df = reshaped_df.groupby(['Model', 'diagnosis', 'label', 'adjective'])['logit'].mean().reset_index()

with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_llms_Meta-Llama-3-8B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model1 = json.load(file)


with open('/Users/ninaliem/thesis11/adj_data/adjective_probabilities_Meta-Llama-2-7B_incremental_NOCON_1610_1.json', 'r') as file:
    nocon_data_model2 = json.load(file)

def aggregate_data_no_diagnosis(data, model):
    reshaped_data = []

    def extract_gender_from_pronouns(prompt):
        pronouns_to_gender = {
            "he": "Male",
            "him": "Male",
            "his": "Male",
            "she": "Female",
            "her": "Female",
            "hers": "Female",
            "they": "Neutral",
            "them": "Neutral",
            "theirs": "Neutral"
        }
        tokens = prompt.lower().split() 
        for pronoun, gender in pronouns_to_gender.items():
            if pronoun in tokens:  
                return gender
        return "Unknown"  

    for prompt, adjectives in data.items():
        gender = extract_gender_from_pronouns(prompt)  

        for adjective, details in adjectives.items():
            reshaped_data.append({
                'Prompt': prompt,
                'adjective': adjective,
                'logit': details["logit"],
                'label': 'negative', 
                'Gender': gender,
                'Model': model,  
                'diagnosis': 'None'  
            })
            
            reshaped_data.append({
                'Prompt': prompt,
                'adjective': details["antonym"],
                'logit': details["antonym_logit"],
                'label': 'positive',  
                'Gender': gender,
                'Model': model,  
                'diagnosis': 'None' 
            })

    return reshaped_data

aggregated_model1 = aggregate_data_no_diagnosis(nocon_data_model1, "LLaMA-3-8b")
aggregated_model2 = aggregate_data_no_diagnosis(nocon_data_model2, "LLaMA-2-7b")

all_nocon = aggregated_model1 + aggregated_model2
nocon_data = pd.DataFrame(all_nocon)

res = pd.concat([reshaped_df, nocon_data], axis=0, ignore_index=True)

In [8]:
temperatures = np.linspace(0.1, 2.0, 20)  # Adjust the number of steps if needed

for temp in temperatures:
    mask_positive = res['label'] == 'positive'
    mask_negative = res['label'] == 'negative'
    
    res.loc[mask_positive, f'logit_T_{temp:.1f}'] = res.loc[mask_positive, 'logit'] / temp
    
    res.loc[mask_negative, f'logit_T_{temp:.1f}'] = res.loc[mask_negative, 'logit']


In [9]:
res

,Prompt,adjective,logit,label,Gender,Model,diagnosis,severity,logit_T_0.1,logit_T_0.2,...,logit_T_1.1,logit_T_1.2,logit_T_1.3,logit_T_1.4,logit_T_1.5,logit_T_1.6,logit_T_1.7,logit_T_1.8,logit_T_1.9,logit_T_2.0
0,She has schizophrenia. She is,violent,3.837729,negative,Female,LLaMA-3-8b,schizophrenia,severe,3.837729,3.837729,...,3.837729,3.837729,3.837729,3.837729,3.837729,3.837729,3.837729,3.837729,3.837729,3.837729
1,She has schizophrenia. She is,nonviolent,5.857870,positive,Female,LLaMA-3-8b,schizophrenia,severe,58.578696,29.289348,...,5.325336,4.881558,4.506054,4.184193,3.905246,3.661169,3.445806,3.254372,3.083089,2.928935
2,She has schizophrenia. She is,dangerous,4.072118,negative,Female,LLaMA-3-8b,schizophrenia,severe,4.072118,4.072118,...,4.072118,4.072118,4.072118,4.072118,4.072118,4.072118,4.072118,4.072118,4.072118,4.072118
3,She has schizophrenia. She is,safe,2.697342,positive,Female,LLaMA-3-8b,schizophrenia,severe,26.973424,13.486712,...,2.452129,2.247785,2.074879,1.926673,1.798228,1.685839,1.586672,1.498524,1.419654,1.348671
4,She has schizophrenia. She is,unfamiliar,6.253426,negative,Female,LLaMA-3-8b,schizophrenia,severe,6.253426,6.253426,...,6.253426,6.253426,6.253426,6.253426,6.253426,6.253426,6.253426,6.253426,6.253426,6.253426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990715,They come across as,real,4.530025,positive,Neutral,LLaMA-2-7b,None,NaN,45.300250,22.650125,...,4.118205,3.775021,3.484635,3.235732,3.020017,2.831266,2.664721,2.516681,2.384224,2.265013
990716,They come across as,dramatic,3.531684,negative,Neutral,LLaMA-2-7b,None,NaN,3.531684,3.531684,...,3.531684,3.531684,3.531684,3.531684,3.531684,3.531684,3.531684,3.531684,3.531684,3.531684
990717,They come across as,unexaggerated,4.961461,positive,Neutral,LLaMA-2-7b,None,NaN,49.614606,24.807303,...,4.510419,4.134550,3.816508,3.543900,3.307640,3.100913,2.918506,2.756367,2.611295,2.480730
990718,They come across as,sick,1.889921,negative,Neutral,LLaMA-2-7b,None,NaN,1.889921,1.889921,...,1.889921,1.889921,1.889921,1.889921,1.889921,1.889921,1.889921,1.889921,1.889921,1.889921


# Log odds ratio


## Scale the logits of the positive adjectives down, to increase their likelihood

In [18]:
aggregated_df = res.groupby(['Model', 'diagnosis', 'Gender', 'label', 'adjective'])['logit_T_0.5'].mean().reset_index()

aggregated_df = aggregated_df[aggregated_df['diagnosis'] != 'a mental illness']

In [20]:
def calculate_log_odds_ratios(group1, group2):
    logits_group1 = group1.set_index('adjective')['logit_T_0.5']
    logits_group2 = group2.set_index('adjective')['logit_T_0.5']
    ratios = {}

    for adj1 in logits_group1.index:
        adj_ratios = []
        for adj2 in logits_group2.index:
            logit1 = logits_group1[adj1]
            logit2 = logits_group2[adj2]
            log_odds_ratio = logit1 - logit2  
            adj_ratios.append(log_odds_ratio)
        ratios[adj1] = np.mean(adj_ratios)  
    return ratios

def analyze_adjectives(aggregated_df):
    log_odds_per_adjective = {}
    grouped = aggregated_df.groupby(['Model', 'diagnosis', 'Gender'])

    for (model, diagnosis, gender), group in grouped:
        negative_model_df = group[group['label'] == 'negative']
        positive_model_df = group[group['label'] == 'positive']
        
        negative_vs_negative = calculate_log_odds_ratios(negative_model_df, negative_model_df)
        negative_vs_positive = calculate_log_odds_ratios(negative_model_df, positive_model_df)
        negative_vs_combined = calculate_log_odds_ratios(negative_model_df, pd.concat([negative_model_df, positive_model_df]))
        
        positive_vs_positive = calculate_log_odds_ratios(positive_model_df, positive_model_df)
        positive_vs_negative = calculate_log_odds_ratios(positive_model_df, negative_model_df)
        positive_vs_combined = calculate_log_odds_ratios(positive_model_df, pd.concat([negative_model_df, positive_model_df]))

        if model not in log_odds_per_adjective:
            log_odds_per_adjective[model] = {}
        if diagnosis not in log_odds_per_adjective[model]:
            log_odds_per_adjective[model][diagnosis] = {}
        log_odds_per_adjective[model][diagnosis][gender] = {
            'negative_vs_negative': negative_vs_negative,
            'negative_vs_positive': negative_vs_positive,
            'negative_vs_combined': negative_vs_combined,
            'positive_vs_positive': positive_vs_positive,
            'positive_vs_negative': positive_vs_negative,
            'positive_vs_combined': positive_vs_combined
        }

    results = {}
    for model in log_odds_per_adjective:
        for diagnosis in log_odds_per_adjective[model]:
            for gender, data in log_odds_per_adjective[model][diagnosis].items():
                key = (model, diagnosis, gender)
                results[key] = pd.DataFrame(data).T
                results[key].index.name = 'Adjective'

    return results

comparison_dfs = analyze_adjectives(aggregated_df)

for (model, diagnosis, gender), df in comparison_dfs.items():
    print(f"\nModel: {model}, Diagnosis: {diagnosis}, Gender: {gender}")
    print(df)



Model: LLaMA-2-7b, Diagnosis: None, Gender: Female
                      abnormal   abusive  aggressive     angry  antisocial  \
Adjective                                                                    
negative_vs_negative  0.273817  0.273817    0.042976 -0.174267   -1.154542   
negative_vs_positive -1.666012 -1.666012   -1.896853 -2.114096   -3.094371   
negative_vs_combined -0.692039 -0.692039   -0.922880 -1.140123   -2.120399   
positive_vs_positive       NaN       NaN         NaN       NaN         NaN   
positive_vs_negative       NaN       NaN         NaN       NaN         NaN   
positive_vs_combined       NaN       NaN         NaN       NaN         NaN   

                      attention-seeking   awkward       bad  blameworthy  \
Adjective                                                                  
negative_vs_negative          -0.968074  0.497444 -0.369018     0.682469   
negative_vs_positive          -2.907903 -1.442385 -2.308846    -1.257359   
negative_vs_combine

In [21]:
def melt_comparison_dfs(comparison_dfs):
    melted_dfs = []  

    for (model, diagnosis, gender), df in comparison_dfs.items():
        positive_indices = ['positive_vs_positive', 'positive_vs_negative', 'positive_vs_combined']
        negative_indices = ['negative_vs_negative', 'negative_vs_positive', 'negative_vs_combined']
        
        df.loc['positive_avg'] = df.loc[positive_indices].mean()
        df.loc['negative_avg'] = df.loc[negative_indices].mean()

        avg_df = df.loc[['positive_avg', 'negative_avg']]
        avg_df.index.name = 'Comparison'

        melted_df = avg_df.reset_index().melt(
            id_vars='Comparison',
            var_name='adjective',
            value_name='average_log_odds_ratio'
        )

        melted_df['model'] = model
        melted_df['diagnosis'] = diagnosis
        melted_df['gender'] = gender

        melted_df = melted_df[melted_df['Comparison'].isin(['positive_avg', 'negative_avg'])]
        
        melted_dfs.append(melted_df)

    final_df = pd.concat(melted_dfs, ignore_index=True)
    return final_df

melted_result = melt_comparison_dfs(comparison_dfs)

print(melted_result)


         Comparison   adjective  average_log_odds_ratio       model  \
0      positive_avg    abnormal                     NaN  LLaMA-2-7b   
1      negative_avg    abnormal               -0.694745  LLaMA-2-7b   
2      positive_avg     abusive                     NaN  LLaMA-2-7b   
3      negative_avg     abusive               -0.694745  LLaMA-2-7b   
4      positive_avg  aggressive                     NaN  LLaMA-2-7b   
...             ...         ...                     ...         ...   
28675  negative_avg       whole                     NaN  LLaMA-3-8b   
28676  positive_avg        wise                2.772284  LLaMA-3-8b   
28677  negative_avg        wise                     NaN  LLaMA-3-8b   
28678  positive_avg   wonderful               -3.007670  LLaMA-3-8b   
28679  negative_avg   wonderful                     NaN  LLaMA-3-8b   

           diagnosis   gender  
0               None   Female  
1               None   Female  
2               None   Female  
3               Non

In [25]:
melted_result.dropna()
melted_result = melted_result.drop(columns=['Comparison'])

# Top adjectives

In [29]:
def get_top_adjectives(melted_results, top_n=30):
    top_adjectives = (
        melted_results
        .groupby(['model', 'diagnosis', 'gender'])
        .apply(lambda group: group.nlargest(top_n, 'average_log_odds_ratio'))
        .reset_index(drop=True)
    )
    return top_adjectives

top_adjectives_result = get_top_adjectives(melted_result, top_n=30)

print(top_adjectives_result)


          adjective  average_log_odds_ratio       model      diagnosis  \
0          innocent                7.581822  LLaMA-2-7b           None   
1     unexaggerated                5.496069  LLaMA-2-7b           None   
2       unmalicious                5.496069  LLaMA-2-7b           None   
3        unneurotic                5.496069  LLaMA-2-7b           None   
4     unproblematic                5.496069  LLaMA-2-7b           None   
...             ...                     ...         ...            ...   
1795          whole                3.321679  LLaMA-3-8b  schizophrenia   
1796          usual                3.314442  LLaMA-3-8b  schizophrenia   
1797       positive                3.303332  LLaMA-3-8b  schizophrenia   
1798      uncapable                3.060772  LLaMA-3-8b  schizophrenia   
1799      uncurable                3.060772  LLaMA-3-8b  schizophrenia   

       gender  
0      Female  
1      Female  
2      Female  
3      Female  
4      Female  
...       ...  

In [32]:
def reshape_top_adjectives(top_adjectives, top_n=30):
    top_adjectives['rank'] = (
        top_adjectives.groupby(['model', 'diagnosis', 'gender'])['average_log_odds_ratio']
        .rank(method='first', ascending=False)
        .astype(int)
    )
    
    top_adjectives = top_adjectives[top_adjectives['rank'] <= top_n]
    
    reshaped_df = top_adjectives.pivot(
        index=['model', 'rank'],
        columns=['diagnosis', 'gender'],
        values='adjective'
    )
    
    reshaped_df = reshaped_df.sort_index(axis=0).sort_index(axis=1)
    
    return reshaped_df

reshaped_results = reshape_top_adjectives(top_adjectives_result, top_n=20)

print(reshaped_results)


diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2     unexaggerated  unexaggerated  unexaggerated   
           3       unmalicious    unmalicious    unmalicious   
           4        unneurotic     unneurotic     unneurotic   
           5     unproblematic  unproblematic  unproblematic   
           6          outgoing       outgoing       outgoing   
           7           serious        serious           good   
           8       intelligent      shameless      excellent   
           9         appealing           good        similar   
           10             sane      confident        serious   
           11          sincere      excellent       inactive   
           12        excellent        capable    incompetent   
           13        confident    intell

### T=0.5

In [33]:
reshaped_results

diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2     unexaggerated  unexaggerated  unexaggerated   
           3       unmalicious    unmalicious    unmalicious   
           4        unneurotic     unneurotic     unneurotic   
           5     unproblematic  unproblematic  unproblematic   
           6          outgoing       outgoing       outgoing   
           7           serious        serious           good   
           8       intelligent      shameless      excellent   
           9         appealing           good        similar   
           10             sane      confident        serious   
           11          sincere      excellent       inactive   
           12        excellent        capable    incompetent   
           13        confident    intelligent       insecure   
           14        shameless      laid-back     nonviolent   
           15           mature      appealing           sane   
           16         flawless         driven        sincere   
           17        wonderful    responsible      shameless   
           18             good         mature      easygoing   
           19      responsible       inactive      appealing   
           20          capable    incompetent        capable   
LLaMA-3-8b 1            strong         strong        healthy   
           2           healthy        healthy         strong   
           3       considerate    considerate    considerate   
           4     compassionate        serious       cheerful   
           5          cheerful       cheerful    transparent   
           6           serious  compassionate        serious   
           7       transparent    transparent  compassionate   
           8      approachable   approachable         secure   
           9         empowered      competent      competent   
           10       comforting     dependable     dependable   
           11         pleasant        assured      empowered   
           12          assured     comforting     comforting   
           13        appealing      appealing          whole   
           14        competent           kind        assured   
           15           serene         secure       pleasant   
           16       dependable       pleasant   approachable   
           17             kind         serene      uncapable   
           18           secure           good      uncurable   
           19            whole        relaxed      organized   
           20          relaxed           wise           good   

diagnosis       antisocial personality disorder                    \
gender                                   Female              Male   
model      rank                                                     
LLaMA-2-7b 1                      unexaggerated     unexaggerated   
           2                        unmalicious       unmalicious   
           3                         unneurotic        unneurotic   
           4                      unproblematic     unproblematic   
           5                       self-assured      self-assured   
           6                           selfless          selfless   
           7                             mature         shameless   
           8                        intelligent          outgoing   
           9                           outgoing            mature   
           10                         shameless       intelligent   
           11                  high-functioning              kind   
           12                       responsible       responsible   
           13                              kind  high-functioning   
           14                        thoughtful        thoughtful   
           15                            social         laid-back   
     

### T = 0.1

In [34]:
aggregated_df = res.groupby(['Model', 'diagnosis', 'Gender', 'label', 'adjective'])['logit_T_0.1'].mean().reset_index()

aggregated_df = aggregated_df[aggregated_df['diagnosis'] != 'a mental illness']

def calculate_log_odds_ratios(group1, group2):
    logits_group1 = group1.set_index('adjective')['logit_T_0.1']
    logits_group2 = group2.set_index('adjective')['logit_T_0.1']
    ratios = {}

    for adj1 in logits_group1.index:
        adj_ratios = []
        for adj2 in logits_group2.index:
            logit1 = logits_group1[adj1]
            logit2 = logits_group2[adj2]
            log_odds_ratio = logit1 - logit2  
            adj_ratios.append(log_odds_ratio)
        ratios[adj1] = np.mean(adj_ratios)  
    return ratios

def analyze_adjectives(aggregated_df):
    log_odds_per_adjective = {}
    grouped = aggregated_df.groupby(['Model', 'diagnosis', 'Gender'])

    for (model, diagnosis, gender), group in grouped:
        negative_model_df = group[group['label'] == 'negative']
        positive_model_df = group[group['label'] == 'positive']
        
        negative_vs_negative = calculate_log_odds_ratios(negative_model_df, negative_model_df)
        negative_vs_positive = calculate_log_odds_ratios(negative_model_df, positive_model_df)
        negative_vs_combined = calculate_log_odds_ratios(negative_model_df, pd.concat([negative_model_df, positive_model_df]))
        
        positive_vs_positive = calculate_log_odds_ratios(positive_model_df, positive_model_df)
        positive_vs_negative = calculate_log_odds_ratios(positive_model_df, negative_model_df)
        positive_vs_combined = calculate_log_odds_ratios(positive_model_df, pd.concat([negative_model_df, positive_model_df]))

        if model not in log_odds_per_adjective:
            log_odds_per_adjective[model] = {}
        if diagnosis not in log_odds_per_adjective[model]:
            log_odds_per_adjective[model][diagnosis] = {}
        log_odds_per_adjective[model][diagnosis][gender] = {
            'negative_vs_negative': negative_vs_negative,
            'negative_vs_positive': negative_vs_positive,
            'negative_vs_combined': negative_vs_combined,
            'positive_vs_positive': positive_vs_positive,
            'positive_vs_negative': positive_vs_negative,
            'positive_vs_combined': positive_vs_combined
        }

    results = {}
    for model in log_odds_per_adjective:
        for diagnosis in log_odds_per_adjective[model]:
            for gender, data in log_odds_per_adjective[model][diagnosis].items():
                key = (model, diagnosis, gender)
                results[key] = pd.DataFrame(data).T
                results[key].index.name = 'Adjective'

    return results

comparison_dfs = analyze_adjectives(aggregated_df)

def melt_comparison_dfs(comparison_dfs):
    melted_dfs = []  

    for (model, diagnosis, gender), df in comparison_dfs.items():
        positive_indices = ['positive_vs_positive', 'positive_vs_negative', 'positive_vs_combined']
        negative_indices = ['negative_vs_negative', 'negative_vs_positive', 'negative_vs_combined']
        
        df.loc['positive_avg'] = df.loc[positive_indices].mean()
        df.loc['negative_avg'] = df.loc[negative_indices].mean()

        avg_df = df.loc[['positive_avg', 'negative_avg']]
        avg_df.index.name = 'Comparison'

        melted_df = avg_df.reset_index().melt(
            id_vars='Comparison',
            var_name='adjective',
            value_name='average_log_odds_ratio'
        )

        melted_df['model'] = model
        melted_df['diagnosis'] = diagnosis
        melted_df['gender'] = gender

        melted_df = melted_df[melted_df['Comparison'].isin(['positive_avg', 'negative_avg'])]
        
        melted_dfs.append(melted_df)

    final_df = pd.concat(melted_dfs, ignore_index=True)
    return final_df

melted_result = melt_comparison_dfs(comparison_dfs)

melted_result.dropna()
melted_result = melted_result.drop(columns=['Comparison'])

def get_top_adjectives(melted_results, top_n=30):
    top_adjectives = (
        melted_results
        .groupby(['model', 'diagnosis', 'gender'])
        .apply(lambda group: group.nlargest(top_n, 'average_log_odds_ratio'))
        .reset_index(drop=True)
    )
    return top_adjectives

top_adjectives_result = get_top_adjectives(melted_result, top_n=30)

def reshape_top_adjectives(top_adjectives, top_n=30):
    top_adjectives['rank'] = (
        top_adjectives.groupby(['model', 'diagnosis', 'gender'])['average_log_odds_ratio']
        .rank(method='first', ascending=False)
        .astype(int)
    )
    
    top_adjectives = top_adjectives[top_adjectives['rank'] <= top_n]
    
    reshaped_df = top_adjectives.pivot(
        index=['model', 'rank'],
        columns=['diagnosis', 'gender'],
        values='adjective'
    )
    
    reshaped_df = reshaped_df.sort_index(axis=0).sort_index(axis=1)
    
    return reshaped_df

reshaped_results = reshape_top_adjectives(top_adjectives_result, top_n=20)

In [35]:
reshaped_results

diagnosis                 None                                \
gender                  Female           Male        Neutral   
model      rank                                                
LLaMA-2-7b 1          innocent       innocent       innocent   
           2     unexaggerated  unexaggerated  unexaggerated   
           3       unmalicious    unmalicious    unmalicious   
           4        unneurotic     unneurotic     unneurotic   
           5     unproblematic  unproblematic  unproblematic   
           6          outgoing       outgoing       outgoing   
           7           serious        serious           good   
           8       intelligent      shameless      excellent   
           9         appealing           good        similar   
           10             sane      confident        serious   
           11          sincere      excellent     nonviolent   
           12        excellent        capable           sane   
           13        confident    intelligent        sincere   
           14        shameless      laid-back      shameless   
           15           mature      appealing      easygoing   
           16         flawless         driven      appealing   
           17        wonderful    responsible        capable   
           18             good         mature         driven   
           19      responsible           sane         mature   
           20          capable        sincere    responsible   
LLaMA-3-8b 1            strong         strong        healthy   
           2           healthy        healthy         strong   
           3       considerate    considerate    considerate   
           4     compassionate        serious       cheerful   
           5          cheerful       cheerful    transparent   
           6           serious  compassionate        serious   
           7       transparent    transparent  compassionate   
           8      approachable   approachable         secure   
           9         empowered      competent      competent   
           10       comforting     dependable     dependable   
           11         pleasant        assured      empowered   
           12          assured     comforting     comforting   
           13        appealing      appealing          whole   
           14        competent           kind        assured   
           15           serene         secure       pleasant   
           16       dependable       pleasant   approachable   
           17             kind         serene      organized   
           18           secure           good           good   
           19            whole        relaxed        relaxed   
           20          relaxed           wise     optimistic   

diagnosis       antisocial personality disorder                    \
gender                                   Female              Male   
model      rank                                                     
LLaMA-2-7b 1                      unexaggerated     unexaggerated   
           2                        unmalicious       unmalicious   
           3                         unneurotic        unneurotic   
           4                      unproblematic     unproblematic   
           5                       self-assured      self-assured   
           6                           selfless          selfless   
           7                             mature         shameless   
           8                        intelligent          outgoing   
           9                           outgoing            mature   
           10                         shameless       intelligent   
           11                  high-functioning              kind   
           12                       responsible       responsible   
           13                              kind  high-functioning   
           14                        thoughtful        thoughtful   
           15                            social         laid-back   
     